In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
import os
from scipy.fftpack import rfft,irfft
sys.path.append(os.path.join(os.getcwd(), ".."))

from metrics.methods import tsd_metrics as TSD
from shared_utils import gen_sde as Gen_dyn

In [ ]:
def system_coordinates_reader(Path_to_data,Attractor_name,num_attractor=0):
    path = Path_to_data+f"/{Attractor_name}_attractors"
    df = pd.read_csv(path + ".csv")
    df_n = df.to_numpy()
    xyzs = df_n[:,1:4]
    t = df_n[:,0]
    return xyzs,t

Path = "/workspaces/ecg_evaluation/results"
xyzs,t = system_coordinates_reader(Path,"lorenz",0)

In [ ]:
def Random_phase_shuffling(signal):
    fft_signal = rfft(signal)
    phase_fs = np.arctan2(fft_signal[2::2],fft_signal[1:-1:2])
    mag = np.sqrt((fft_signal[1:-1:2])**2+(fft_signal[2::2])**2)
    rng_phase = phase_fs.copy()
    np.random.shuffle(rng_phase)
    fsrp = mag[:, np.newaxis] * np.c_[np.cos(rng_phase), np.sin(rng_phase)]
    fsrp = np.r_[fft_signal[0], fsrp.ravel(), fft_signal[-1]]
    tsr = irfft(fsrp)
    return tsr

In [ ]:
##Adding noise yith specific SNR : 

def add_observational_noise(sig,SNR):
    Power_sig = np.mean(np.abs(sig)**2)
    P_db = 10*np.log10(Power_sig)
    noisedb = P_db - SNR
    sd_db_watts = 10**(noisedb/10)
    #sd_noise = np.sqrt(Power_sig/(SNR))
    noise = np.random.normal(0,np.sqrt(sd_db_watts),len(sig))
    sig_noisy = sig+noise
    return sig_noisy

def add_observational_noise_val(sig,SNR):
    Power_sig = np.mean(np.abs(sig)**2)
    P_db = 10*np.log10(Power_sig)
    noisedb = P_db - SNR
    sd_db_watts = 10**(noisedb/10)
    #sd_noise = np.sqrt(Power_sig/(SNR))
    noise = np.random.normal(0,np.sqrt(sd_db_watts),(len(sig),3))
    #sig_noisy = sig+noise
    return noise

In [ ]:
##TSD code : 
def Lm_q(signal,m,k,fs):
    N = len(signal)
    n = np.floor((N-m)/k).astype(np.int64)
    norm  = ((N-1)/(n*k*(1/fs)))
    sum = np.sum(np.abs(np.diff(signal[m::k], n=1)))
    Lmq = (sum*norm)/k
    return Lmq

def Lq_k(signal,k,fs):
    calc_L_series = np.frompyfunc(lambda m: Lm_q(signal,m,k,fs), 1, 1)
    L_average = np.average(calc_L_series(np.arange(1, k+1)))
    return L_average


def Dq(signal,kmax,fs):
    calc_L_average_series = np.frompyfunc(lambda k: Lq_k(signal,k,fs), 1, 1)

    k = np.arange(1, kmax+1)
    L = calc_L_average_series(k).astype(np.float64)

    D, _ = - np.polyfit(np.log2(k), np.log2(L), 1)

    return D

In [ ]:
###Time for Dynamical noise : 
##For simplifying the workflow, we will do ourselves the generation of a lorenz and rossler system at any noise level 
##Noise level definition is the same as the one use by the article

def compute_attractor_3d(attractor, x0, y0, z0, dt, num_steps,sigma_array = np.array([0,0,0]),noise_lev = 0):
    # Step through "time", calculating the partial derivatives
    # at the current point and using them to estimate the next point
    # Need one more for the initial values

    xs = np.empty(num_steps + 1)
    ys = np.empty(num_steps + 1)
    zs = np.empty(num_steps+ 1)
    xs[0] = x0
    ys[0] = y0
    zs[0] = z0
    t = np.empty(num_steps + 1)
    t[0] = 0.0
    if noise_lev ==0:
        dWq = np.zeros((num_steps,3))
        noise_w = False
    else : 
        dWq = add_observational_noise_val(xyzs[:,0].copy(),noise_lev)
        noise_w = True
    for i in range(num_steps):
        dar = attractor(t[i],np.array([xs[i],ys[i],zs[i]]),num_steps,dWq[i,:].copy(),sigma_w = sigma_array.copy(),l_noise=noise_lev,w_dnoise = noise_w,dt1 = dt)
        xs[i + 1] = xs[i] + (dar[0] * dt)
        ys[i + 1] = ys[i] + (dar[1] * dt)
        zs[i + 1] = zs[i] + (dar[2] * dt)
        t[i + 1] = (i + 1) * dt

    coordinates = np.empty((len(xs),3))
    coordinates[:,0] = xs
    coordinates[:,1] = ys
    coordinates[:,2] = zs
    return coordinates, t



In [ ]:
###Attractor used : Lorenz and Rossler (same as article. you can add more attractor if you want)
def lorenz(t,state,number_step,dW, sigma=10.0, beta=8 / 3, rho=28.0,sigma_w = np.array([0,0,0]),l_noise = 0, w_dnoise=False, dt1=0.001):
    """
    Lorenz attractor.
    Lorenz attractor is ordinary differential equation (ODE) of 3rd
    order system. In 1963, E. Lorenz developed a simplified mathematical
    model for atmospheric convection.

    Parameters
    ==========
    sigma, beta, rho - are lorenz system parameters.
    Default values are:
            - x0 = 0, y0 = 1, z0 = 1.05, sigma = 10, beta = 8/3, rho = 28
    """
    if w_dnoise and len(sigma_w) !=0 and l_noise !=0:
        dWx,dWy,dWz = dW[0],dW[1],dW[2]
        rhox= 1#sigma_w[0]*l_noise
        rhoy= 1#sigma_w[1]*l_noise
        rhoz =1 # sigma_w[2]*l_noise
    else:
        dWx,dWy,dWz = 0, 0, 0
        rhox, rhoy, rhoz = 0, 0, 0
    x, y, z = state

    xdot = (rho*y - sigma*x) + rhox * dWx
    ydot = x * (rho - z) - y + rhoy * dWy
    zdot = x * y - beta * z + rhoz * dWz
    return np.array([xdot, ydot, zdot])

def rossler(t,state,number_step,dW, a=0.36, b=0.4, c=4.5,sigma_w = np.array([0,0,0]),l_noise = 0, w_dnoise=False, dt1=0.001):
    """
    Rossler attractor.

    Parameters
    ==========
    a, b and c - are rossler system parameters.
    Default values are:
            - x0 = 0, y0 = 0, z0 = 0, a = 0.2, b = 0.2 and c = 5.7.

    Other useful combinations are:
    1) x0 = 0, y0 = 0, z0 = 0, a = 0.1, b = 0.1 and c = 14 (another useful parameters)
    2) x0 = 0, y0 = 0, z0 = 0, a = 0.5, b = 1.0 and c = 3 (J. C. Sprott)

    Notes
    =====
    - Varying a:
    b = 0.2 and c = 5.7 are fixed. Change a:

    a <= 0      : Converges to the centrally located fixed point
    a = 0.1     : Unit cycle of period 1
    a = 0.2     : Standard parameter value selected by Rössler, chaotic
    a = 0.3     : Chaotic attractor, significantly more Möbius strip-like
                              (folding over itself).
    a = 0.35    : Similar to .3, but increasingly chaotic
    a = 0.38    : Similar to .35, but increasingly chaotic

    - Varying b:
    a = 0.2 and c = 5.7 are fixed. Change b:

    If b approaches 0 the attractor approaches infinity, but if b would
    be more than a and c, system becomes not a chaotic.

    - Varying c:
    a = b = 0.1 are fixed. Change c:

    c = 4       : period-1 orbit,
    c = 6       : period-2 orbit,
    c = 8.5     : period-4 orbit,
    c = 8.7     : period-8 orbit,
    c = 9       : sparse chaotic attractor,
    c = 12      : period-3 orbit,
    c = 12.6    : period-6 orbit,
    c = 13      : sparse chaotic attractor,
    c = 18      : filled-in chaotic attractor.
    """

    if w_dnoise and len(sigma_w) !=0 and l_noise !=0:
        dWx,dWy,dWz = dW[0],dW[1],dW[2]
        rhox= 1#sigma_w[0]*l_noise
        rhoy= 1#sigma_w[1]*l_noise
        rhoz =1 # sigma_w[2]*l_noise
    else:
        dWx,dWy,dWz = dW[0],dW[1],dW[2]
        rhox, rhoy, rhoz = 0, 0, 0
    x, y, z = state
    xdot = -(y + z) + rhox * dWx
    ydot = x + a * y + rhoy * dWy
    zdot = b + z * (x - c) + rhoz * dWz
    return np.array([xdot, ydot, zdot])   

In [ ]:
dict_attractor = {"lorenz":lorenz,"rossler":rossler}
##Following the study : 
noise_level = np.linspace(-10,100,10)
##Noise the initial condition value
high_noise = {
    "lorenz": 1.0,
    "rossler": 1.0,
}

name_attractors = ["lorenz","rossler"]

##Series generator:

def Series_generator(der_attractor,name_attractor,start_index,num_sim,sigma_w = np.array([0,0,0]),noise_l = 0):
    ##Initial condition
    w = np.random.uniform(low = 0,high = high_noise[name_attractor])
    x0 = 0.6+w
    y0 = 0.2+w
    z0 = 0.1+w
    dt = 0.001
    num_steps = 35000
    coordinates,t = compute_attractor_3d(der_attractor,x0,y0,z0,dt,num_steps,sigma_array=sigma_w,noise_lev=noise_l)
    sigma_ar = np.array([np.std(coordinates[start_index:,0]),np.std(coordinates[start_index:,1]),np.std(coordinates[start_index:,2])])
    return coordinates[start_index:,:],t[start_index:],sigma_ar

In [ ]:
#Create the function that does TSD vs noise level for dynamical noise:
distance={"lorenz":127,"rossler":229}


def TSDvsNoiseLevel_dyn(dico_attractor,name_attractor,level_noise,number_simulation):
    Dmean = {}
    SD_D = {}
    sigma_clean = {}
    for i in level_noise:
        if i ==0:
            for n in name_attractor:
                mid_Dmean = np.array([])
                mid_SD = np.array([])
                int_sigma_clean = np.zeros((number_simulation,3))
                for j in range(number_simulation):
                    coordinates_clean,t_clean,sigma_sim = Series_generator(dico_attractor[n],n,3000,j)
                    Obs = coordinates_clean[:,0].copy()
                    Mean_TSD,SD_TSD = TSD.TSD_mean_calculator(Obs,distance[n])
                    mid_Dmean = np.append(mid_Dmean,Mean_TSD)
                    mid_SD = np.append(mid_SD,SD_TSD)
                    int_sigma_clean[j,:] = sigma_sim
                sigma_clean[n] = np.array([np.mean(int_sigma_clean[:,0]),np.mean(int_sigma_clean[:,1]),np.mean(int_sigma_clean[:,2])])
                Dmean[n] = np.array([np.mean(mid_Dmean)])
                SD_D[n] = np.array([np.mean(mid_SD)])
        else : 
            for n in name_attractor:
                mid_Dmean = np.array([])
                mid_SD = np.array([])
                for j in range(1,number_simulation):
                    coordinates_clean,_,_ = Series_generator(dico_attractor[n],n,3000,j,sigma_w = sigma_clean[n],noise_l = i)
                    Obs = coordinates_clean[:,0].copy()
                    Mean_TSD,SD_TSD = TSD.TSD_mean_calculator(Obs,distance[n])
                    mid_Dmean = np.append(mid_Dmean,Mean_TSD)
                    mid_SD = np.append(mid_SD,SD_TSD)
                Dmean[n] = np.append(Dmean[n],np.mean(mid_Dmean))
                SD_D[n] = np.append(SD_D[n],np.mean(mid_SD))
    return Dmean,SD_D

def plt_TSDvsdyn_Noise(dico_attractor,noise_lev,attractors_sel,n_simulation):
    Great_mean,Great_SD= TSDvsNoiseLevel_dyn(dico_attractor,attractors_sel,noise_lev,n_simulation)
    fig,ax = plt.subplots(len(attractors_sel)-1,2,figsize = (20,10))
    for i,j in zip(attractors_sel,range(len(attractors_sel))):
        #ax[j].plot(noise_lev,Great_mean[i],"ob")
        ax[j].errorbar(noise_lev,Great_mean[i],Great_SD[i],fmt = "o",color='red',
             ecolor='black', elinewidth=3, capsize=0)
        ax[j].set_xlabel("Noise level")
        ax[j].set_ylabel("mean TSD value")
        ax[j].set_title(f"TSD vs noise level for {i} system with Dynamical noise")
        ax[j].grid()


    plt.figure()
    for i in attractors_sel:
        plt.plot(noise_lev,Great_mean[i])
    plt.legend([i for i in attractors_sel])
    plt.title("Mean TSD value evolution with noise level for both system with Dynamical noise")
    plt.xlabel("Noise level")
    plt.ylabel("mean TSD value")
    plt.grid()
    plt.show()
                
plt_TSDvsdyn_Noise(dict_attractor,noise_level,name_attractors,100)


In [ ]:
dict_attractor = {"lorenz":lorenz,"rossler":rossler}
name_attractors = ["lorenz","rossler"]
dico_sys = {}
for i in name_attractors:
    xyzs,t,_ = Series_generator(dict_attractor[i],i,3000,1)
    dico_sys[i] = xyzs[:,0].copy()
fs_l = 1/(t[1]-t[0])
plt.plot(t,TSD.RMS_array_creator(dico_sys,name_attractors,t,fs_l))
plt.title("Evolution of RMS value function of length of time series")
plt.xlabel("segment length")
plt.ylabel("RMS val")
plt.grid()
plt.show()

arr = TSD.RMS_array_creator(dico_sys,name_attractors,t,fs_l)
print(arr[~np.isnan(arr)])
index_min = np.argmin(arr[~np.isnan(arr)])

print(index_min)
print(arr[index_min])
print("Behold! c* accoridng to RMS :",t[index_min])

In [ ]:
###shuffling time!!!!!
def The_Big_shufflers(xyzs_clean):
    xyzs_shuffled=np.empty((xyzs_clean.shape[0]+1,3))
    x = xyzs[:,0]
    y = xyzs[:,1]
    z = xyzs[:,2]
    x_noisy = Random_phase_shuffling(x)
    y_noisy = Random_phase_shuffling(y)
    z_noisy = Random_phase_shuffling(z)
    xyzs_shuffled[:,0] = x_noisy
    xyzs_shuffled[:,1] = y_noisy
    xyzs_shuffled[:,2] = z_noisy
    return xyzs_shuffled


In [ ]:
###Put dynamical noise in the system :

# system      = 'lorenz_stochastic'
# p           = (10.0, 8.0/3.0, 28.0)
system      = 'rossler_stochastic'
p           = (0.2, 0.2, 5.7)
observables = [0,1,2]
length      = 10000
x0          = None
step        = 0.001
sample      = 0.01
discard     = 1000
epsilon_sde = 0.0
epsilon_add = 0.0
beta_add    = 0.0

t_obs, X_obs, t_gen, X_gen = Gen_dyn._gen_data(
			system=system,
			observables=observables,
			length=length,
			x0=x0,
			p=p, step=step,
			sample=sample,
			discard=discard,
			epsilon_sde=epsilon_sde,
			epsilon_add=epsilon_add,
			beta_add=beta_add
	)

print(X_obs.shape)
print(X_gen.shape)
plt.figure()
ax1 = plt.subplot(311)
ax1.plot(t_obs,X_obs[:,0],'.k')
ax1.plot(t_gen,X_gen[:,0],'.r')
ax2 = plt.subplot(312, sharex=ax1)
ax2.plot(t_obs,X_obs[:,1],'.k')
ax2.plot(t_gen,X_gen[:,1],'.r')
ax3 = plt.subplot(313, sharex=ax1)
ax3.plot(t_obs,X_obs[:,2],'.k')
ax3.plot(t_gen,X_gen[:,2],'.r')

fig = plt.figure(figsize = (15,15))

ax = fig.add_subplot(111, projection='3d')
fig.tight_layout(h_pad=7)
ax.set_title("Rossler System")
ax.plot(X_obs[:,0],X_obs[:,1],X_obs[:,2],'.k')
ax.plot(X_gen[:,0],X_gen[:,1],X_gen[:,2],'.r')
ax.set_xlabel("X",fontsize = 20.0)
ax.set_ylabel("Y",fontsize = 20.0)
ax.set_zlabel("Z",fontsize = 20.0)
plt.show()

In [ ]:
###Matrix creator
###Matrix created : n*m (n = Observational noise level added; m = dynamical noise level added)
###For 1 noise level, we add to the signal a specific amount of Observational noise and see how it goes


##Get SD of clean system for x component of each system : 
list_attractor = np.array(["lorenz","rossler"])

SD_sys = {}
P_sys = {list_attractor[0] : (10.0, 8.0/3.0, 28.0),list_attractor[1] :(0.2, 0.2, 5.6)}
print(P_sys)
w = np.random.uniform(low = 0.01,high = 1)
#X0 = {list_attractor[0]:(0.6+w,0.2+w,0.1+w) ,list_attractor[1] : (0.5+w,0.3+w,0.4+w) }
for i in list_attractor:
	system      = f'{i}_stochastic'
	p = P_sys[i]
	observables = [0,1,2]
	length      = 10000
	x0          = None
	step        = 0.001
	sample      = 0.01
	discard     = 1000
	epsilon_sde = 0.0
	epsilon_add = 0.0
	beta_add    = 0.0
	t_obs, X_obs, t_gen, X_gen = Gen_dyn._gen_data(
			system=system,
			observables=observables,
			length=length,
			x0=x0,
			p=p, step=step,
			sample=sample,
			discard=discard,
			epsilon_sde=epsilon_sde,
			epsilon_add=epsilon_add,
			beta_add=beta_add
	)
	sig_X,sig_Y,sig_Z = np.std(X_gen[:,0]),np.std(X_gen[:,1]),np.std(X_gen[:,2])
	SD_sys[i] = (sig_X,sig_Y,sig_Z)
print(SD_sys)


In [ ]:


def TSD_mean_calculator(signal,segment_length, dt=0.01):
    X = np.c_[[signal[int((w - 1)) : int((w) + segment_length)] for w in range(1, int(len(signal) - segment_length))]]
    L1 = np.array([TSD.Lq_k(X[i, :], 1, 1 / dt) for i in range(X.shape[0])])
    L2 = np.array([TSD.Lq_k(X[i, :], 2, 1 / dt) for i in range(X.shape[0])])
    Ds = (np.log(L1) - np.log(L2)) / (np.log(2))
    return np.mean(Ds)

def TSD_mean_calculator_art(signal,segment_length, dt=0.01):
    X = np.c_[[signal[int((w - 1)) : int((w) + segment_length)] for w in range(1, int(len(signal) - segment_length))]]
    L1 = np.array([TSD.Lq_k(X[i, :], 1, 1 / dt) for i in range(X.shape[0])])
    L2 = np.array([TSD.Lq_k(X[i, :], 2, 1 / dt) for i in range(X.shape[0])])
    Ds = (np.log(L1) - np.log(L2)) / (np.log(2))
    return np.mean(Ds),np.percentile(Ds,25),np.percentile(Ds,75)


In [ ]:
###Noise lets fill the matrix : We will only do for the x component


epsilon_add = 0.0
beta_add    = 0.0
x0 = None
observables = [0,1,2]
length = 10000
step = 0.001
sample = 0.01
discard = 1000
size_ar = 20
Obs_noise = np.linspace(-10,100,size_ar)
Dynamical_noise = np.linspace(0,1,size_ar)
M_TSD = {list_attractor[0] : np.empty([size_ar,size_ar]),list_attractor[1]: np.empty([size_ar,size_ar])}
seg_attractor = {list_attractor[0] : 127,list_attractor[1] : 127}

Mat_dyn_noise =  {list_attractor[0] : np.empty([length,size_ar]),list_attractor[1]: np.empty([length,size_ar])}
for attractor in list_attractor:
    for dyn,col_ind in zip(Dynamical_noise,range(len(Dynamical_noise))):
        system= f'{attractor}_stochastic'
        p = P_sys[attractor]
        epsilon_sde = SD_sys[attractor][0]*dyn
        
        t_obs, X_obs, t_gen, X_gen = Gen_dyn._gen_data(system=system,
            observables=observables,
			length=length,
			x0=x0,
			p=p, step=step,
			sample=sample,
			discard=discard,
			epsilon_sde=epsilon_sde,
			epsilon_add=epsilon_add,
			beta_add=beta_add
        )
        Mat_dyn_noise[attractor][:,col_ind] = X_obs[:,0]



for attractor in list_attractor:
    for col_ind in range(Mat_dyn_noise[attractor].shape[1]):
        sig_interest = Mat_dyn_noise[attractor][:,col_ind]
        stack_noise = (add_observational_noise(sig_interest,observational) for observational in Obs_noise)
        X_noise = np.vstack(list(stack_noise))
        TSD_val_arr = np.array([TSD_mean_calculator(X_noise[ind,:],seg_attractor[attractor],sample) for ind in range(X_noise.shape[0])])
        M_TSD[attractor][:,col_ind] = TSD_val_arr
    print(f"TSD matrix done for {attractor} : ")


In [ ]:
print(M_TSD)

In [ ]:
##Since now we have access to each signal, we can do the article plot (TSD VS dynamical noise)

TSD_val = {list_attractor[0] : np.array([]),list_attractor[1]: np.array([])}
TSD_SD_val = {list_attractor[0] : np.empty([2,size_ar]),list_attractor[1]:np.empty([2,size_ar])}


for att in list_attractor:
    for j in range(Mat_dyn_noise[att].shape[1]):
        m,p25,p75 = TSD_mean_calculator_art(Mat_dyn_noise[att][:,j],seg_attractor[att],sample)
        TSD_val[att] = np.append(TSD_val[att],m)
        TSD_SD_val[att][:,j] = np.array([np.abs(m-p25),np.abs(m-p75)])  

labels = []
for i in list_attractor:
    plt.errorbar(Dynamical_noise,TSD_val[i],TSD_SD_val[i])
        #plt.plot(noise_level,BDM[i])
    labels.append(i)
plt.legend(labels, ncol=4, loc='best',  
           columnspacing=1.0, labelspacing=0.0,
           handletextpad=0.0, handlelength=1.5,
           fancybox=True, shadow=True)
    
plt.xlabel("Dynamical noise")
plt.ylabel("mean TSD value")
plt.title(f"TSD vs Dynamical noise for average TSD value,for both system") 
plt.grid()
plt.show()


In [ ]:
from matplotlib.ticker import MaxNLocator
plt.figure(figsize=(10, 10), dpi=80)
plt.rcParams.update({'font.size':20})
plt.imshow(M_TSD[list_attractor[0]])
plt.xticks(range(len(Dynamical_noise)),["{:.2f}".format(j) for j in Dynamical_noise])
plt.yticks(range(len(Obs_noise)),["{:.2f}".format(j) for j in Obs_noise])
plt.gca().xaxis.set_major_locator(MaxNLocator(prune='lower'))
plt.title(f"TSD evolution for different observational and dynamical noise levels for the {list_attractor[0]}")
plt.xlabel("Dynamical Noise Level")
plt.ylabel("Observational SNR level (dB)")
plt.colorbar(fraction=0.046, pad=0.04)
plt.show()